In [2]:
import keras

Using TensorFlow backend.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
encoding = 'unicode_escape'

file_id = '1PIKSNOsgJ_J9QQBnPUJ759sor9cM2p88'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.zip') 
!unzip data.zip

In [0]:
import cv2

In [0]:
import glob

In [9]:
path='/content/test_set/test_set/cats/cat.4001.jpg'
cv2.imread(path).shape

(415, 498, 3)

In [0]:
path='/content/training_set/training_set/dogs/*.jpg'
files=glob.glob(path)
for i in files:
  #print(i)
  #break
  img=cv2.imread(i)
  #print(img)
  #break
  img=cv2.resize(img,(50,50))
  cv2.imwrite(i,img)

In [30]:
path='/content/training_set/training_set/dogs/dog.1001.jpg'
cv2.imread(path).shape

(50, 50, 3)

In [0]:
path='/content/training_set/training_set/cats/*.jpg'
files=glob.glob(path)
for i in files:
  #print(i)
  #break
  img=cv2.imread(i)
  #print(img)
  #break
  img=cv2.resize(img,(50,50))
  cv2.imwrite(i,img)

In [0]:
path='/content/test_set/test_set/cats/*.jpg'
files=glob.glob(path)
for i in files:
  #print(i)
  #break
  img=cv2.imread(i)
  #print(img)
  #break
  img=cv2.resize(img,(50,50))
  cv2.imwrite(i,img)

In [0]:
path='/content/test_set/test_set/dogs/*jpg'
files=glob.glob(path)
for i in files:
  #print(i)
  #break
  img=cv2.imread(i)
  #print(img)
  #break
  img=cv2.resize(img,(50,50))
  cv2.imwrite(i,img)

In [41]:
from keras import optimizers
from keras.models import Model,Sequential
from keras.layers import Dense, Conv2D, Flatten
classifier=Sequential()
classifier.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(50,50,3)))
#model.add(Conv2D(32, kernel_size=3, activation=’relu’))
classifier.add(Conv2D(16, kernel_size=3, activation='relu'))
#model.add(Conv2D(8, kernel_size=3, activation=’relu’))
#model.add(Conv2D(4, kernel_size=3, activation=’relu’))
classifier.add(Flatten())
classifier.add(Dense(2, activation='softmax'))
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 16)        9232      
_________________________________________________________________
flatten_1 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 67714     
Total params: 78,738
Trainable params: 78,738
Non-trainable params: 0
_________________________________________________________________


In [45]:
from keras.optimizers import Adam

adam = Adam(epsilon = 0.01)
classifier.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)

#print(test_datagen)

train_folder = '/content/training_set/training_set' 
test_folder = '/content/test_set/test_set'


train_set = train_datagen.flow_from_directory(train_folder,
                                              classes=['dogs','cats'],
                                              target_size=(50, 50),
                                              batch_size=32,class_mode='categorical',
                                              color_mode='rgb',
                                              seed=5)

test_set = test_datagen.flow_from_directory(test_folder,
                                            classes=['dogs','cats'],
                                            target_size=(50, 50),
                                            batch_size=32,
                                            class_mode='categorical',
                                            color_mode='rgb',
                                            seed=5)




Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [46]:
history = classifier.fit_generator(train_set,steps_per_epoch=1200,epochs=5,validation_data=test_set,validation_steps=300)

Epoch 1/5
1200/1200 [==============================] - 316s 263ms/step - loss: 0.6268 - accuracy: 0.6419 - val_loss: 0.6352 - val_accuracy: 0.6625
Epoch 2/5
1200/1200 [==============================] - 310s 258ms/step - loss: 0.4914 - accuracy: 0.7592 - val_loss: 0.6386 - val_accuracy: 0.6855
Epoch 3/5
1200/1200 [==============================] - 316s 263ms/step - loss: 0.3358 - accuracy: 0.8544 - val_loss: 0.6943 - val_accuracy: 0.6925
Epoch 4/5
1200/1200 [==============================] - 312s 260ms/step - loss: 0.1748 - accuracy: 0.9354 - val_loss: 1.5303 - val_accuracy: 0.6903
Epoch 5/5
1200/1200 [==============================] - 317s 264ms/step - loss: 0.0556 - accuracy: 0.9876 - val_loss: 1.5097 - val_accuracy: 0.6986
